# Preliminary tests about the decoding accuracy, etc.

This notebook contains several small tests of the decoding accuracy of various decoders to help me understand the pros and cons.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qec_lego_bench.cli.logical_error_rate import *
from qec_lego_bench.stats import Stats
import os

num_workers = os.cpu_count() - 1
print(f"using {num_workers} workers in simulation")
def resume_filepath(name: str) -> str:
    return f"preliminary-decoder-compare-{name}.csv"

global_kwargs = dict(    
    max_shots = 1_000_000_000,
    max_errors = 10_000,
    num_workers = num_workers,
    no_print = True,
)

using 9 workers in simulation


## Bivariate Bicycle Code $[[72, 12, 6]]$ with $p = 0.01$ depolarizing noise on data qubits

In [3]:
default_kwargs = {
    **global_kwargs,
    **dict(
        code = "bb(n=72,k=12,d=6)",
        noise = "depolarize(p=0.01)",
        save_resume_filepath = resume_filepath("bb"),
    )
}
custom_kwargs = [
    dict(decoder="mwpf", max_errors=4000),
    dict(decoder="huf"),
    dict(decoder="mwpf(cluster_node_limit=1000)", max_errors=1000),
    dict(decoder="bposd"),
    dict(decoder="bposd(max_iter=5)"),
    dict(decoder="bposd(max_iter=5,osd_order=10,osd_method=osd_e)", max_errors=1000),
    dict(decoder="bplsd"),
    dict(decoder="bplsd(max_iter=5)", max_errors=1000),
    dict(decoder="bpuf", max_errors=4000),
    dict(decoder="bpuf(max_iter=5)", max_errors=4000),
]

for custom in custom_kwargs:
    kwargs = { **default_kwargs, **custom }
    stats = logical_error_rate(**kwargs)
    print(f"{kwargs['decoder']}: {stats}, duration = {stats.duration // 60} min")

/Users/wuyue/Documents/GitHub/qec-lego-bench/src/qec_lego_bench/cli/logical_error_rate.py:70: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # type: ignore


mwpf: Stats{ pL = 4291/105863577= 4.1(2)e-05, speed=9.74e-05s/S }, duration = 171.0 min
huf: Stats{ pL = 10035/146322841= 6.9(2)e-05, speed=3.34e-05s/S }, duration = 81.0 min
mwpf(cluster_node_limit=1000): Stats{ pL = 1177/30319001= 3.9(3)e-05, speed=1.31e-04s/S }, duration = 66.0 min
bposd: Stats{ pL = 11023/1312153= 0.0084(2), speed=5.55e-05s/S }, duration = 1.0 min
bposd(max_iter=5): Stats{ pL = 11600/133337497= 8.7(2)e-05, speed=3.66e-05s/S }, duration = 81.0 min
bposd(max_iter=5,osd_order=10,osd_method=osd_e): Stats{ pL = 1023/26121625= 3.9(3)e-05, speed=2.31e-04s/S }, duration = 100.0 min
bplsd: Stats{ pL = 12722/1524121= 0.0083(2), speed=6.53e-05s/S }, duration = 1.0 min
bplsd(max_iter=5): Stats{ pL = 1054/12387737= 8.5(7)e-05, speed=4.13e-05s/S }, duration = 8.0 min
bpuf: Stats{ pL = 4101/30652825= 0.000134(5), speed=6.43e-05s/S }, duration = 32.0 min
bpuf(max_iter=5): Stats{ pL = 4392/57984409= 7.6(3)e-05, speed=4.74e-05s/S }, duration = 45.0 min


## Circuit-level Rotated Surface Code, $d=3, r=3, p=0.001$

In [4]:
default_kwargs = {
    **global_kwargs,
    **dict(
        code = CodeCli("rsc(d=3,p=0.001)"),
        max_errors = 10000,
        save_resume_filepath = resume_filepath("rsc"),
    )
}
custom_kwargs = [
    dict(decoder="mwpf"),
    dict(decoder="huf"),
    dict(decoder="mwpf(cluster_node_limit=1000)"),
    dict(decoder="bposd"),
    dict(decoder="bposd(max_iter=5)"),
    dict(decoder="bposd(max_iter=5,osd_order=10,osd_method=osd_e)"),
    dict(decoder="bplsd"),
    dict(decoder="bplsd(max_iter=5)"),
    dict(decoder="bpuf"),
    dict(decoder="bpuf(max_iter=5)"),
    dict(decoder="pymatching", max_errors=1000000),
]

for custom in custom_kwargs:
    kwargs = { **default_kwargs, **custom }
    stats = logical_error_rate(**kwargs)
    print(f"{kwargs['decoder']}: {stats}, duration = {stats.duration // 60} min")

mwpf: Stats{ pL = 13458/21087505= 0.00064(1), speed=3.40e-05s/S }, duration = 11.0 min
huf: Stats{ pL = 11067/15830425= 0.00070(2), speed=1.63e-05s/S }, duration = 4.0 min
mwpf(cluster_node_limit=1000): Stats{ pL = 12646/19796377= 0.00064(1), speed=3.63e-05s/S }, duration = 11.0 min
bposd: Stats{ pL = 13572/18673049= 0.00073(2), speed=3.83e-05s/S }, duration = 11.0 min
bposd(max_iter=5): Stats{ pL = 11744/16232857= 0.00072(2), speed=1.57e-05s/S }, duration = 4.0 min
bposd(max_iter=5,osd_order=10,osd_method=osd_e): Stats{ pL = 11100/17798553= 0.00062(2), speed=4.03e-05s/S }, duration = 11.0 min
bplsd: Stats{ pL = 10384/14387609= 0.00072(2), speed=5.01e-05s/S }, duration = 12.0 min
bplsd(max_iter=5): Stats{ pL = 12762/17908121= 0.00071(2), speed=2.84e-05s/S }, duration = 8.0 min
bpuf: Stats{ pL = 11865/14052761= 0.00084(2), speed=5.12e-05s/S }, duration = 11.0 min
bpuf(max_iter=5): Stats{ pL = 10779/12746137= 0.00085(2), speed=2.85e-05s/S }, duration = 6.0 min
pymatching: Stats{ pL = 120

## Circuit-level Unrotated Surface Code, $d=3, r=3, p=0.001$

In [5]:
default_kwargs = {
    **global_kwargs,
    **dict(
        code = CodeCli("usc(d=3,p=0.001)"),
        max_errors = 10000,
        save_resume_filepath = resume_filepath("usc"),
    )
}
custom_kwargs = [
    dict(decoder="mwpf"),
    dict(decoder="huf"),
    dict(decoder="mwpf(cluster_node_limit=1000)"),
    dict(decoder="bposd"),
    dict(decoder="bposd(max_iter=5)"),
    dict(decoder="bposd(max_iter=5,osd_order=10,osd_method=osd_e)"),
    dict(decoder="bplsd"),
    dict(decoder="bplsd(max_iter=5)"),
    dict(decoder="bpuf"),
    dict(decoder="bpuf(max_iter=5)"),
    dict(decoder="pymatching", max_errors=1000000),
]

for custom in custom_kwargs:
    kwargs = { **default_kwargs, **custom }
    stats = logical_error_rate(**kwargs)
    print(f"{kwargs['decoder']}: {stats}, duration = {stats.duration // 60} min")

mwpf: Stats{ pL = 10876/21749145= 0.00050(1), speed=9.14e-05s/S }, duration = 33.0 min
huf: Stats{ pL = 10043/18809241= 0.00053(1), speed=2.71e-05s/S }, duration = 8.0 min
mwpf(cluster_node_limit=1000): Stats{ pL = 13000/25708185= 0.00051(1), speed=1.52e-04s/S }, duration = 65.0 min
bposd: Stats{ pL = 13651/21214617= 0.00064(1), speed=1.31e-04s/S }, duration = 46.0 min
bposd(max_iter=5): Stats{ pL = 13095/19354009= 0.00068(2), speed=2.62e-05s/S }, duration = 8.0 min
bposd(max_iter=5,osd_order=10,osd_method=osd_e): Stats{ pL = 12906/23934361= 0.00054(1), speed=8.27e-05s/S }, duration = 32.0 min
bplsd: Stats{ pL = 11858/18024857= 0.00066(2), speed=1.55e-04s/S }, duration = 46.0 min
bplsd(max_iter=5): Stats{ pL = 10545/15582617= 0.00068(2), speed=4.62e-05s/S }, duration = 11.0 min
bpuf: Stats{ pL = 13112/17836441= 0.00074(2), speed=1.56e-04s/S }, duration = 46.0 min
bpuf(max_iter=5): Stats{ pL = 11286/15084953= 0.00075(2), speed=4.76e-05s/S }, duration = 11.0 min
pymatching: Stats{ pL = 8

## Circuit-level Repetition Code, $d=3, r=3, p=0.001$

In [6]:
default_kwargs = {
    **global_kwargs,
    **dict(
        code = CodeCli("rep(d=3,p=0.001)"),
        max_errors = 10000,
        save_resume_filepath = resume_filepath("rep"),
    )
}
custom_kwargs = [
    dict(decoder="mwpf"),
    dict(decoder="huf"),
    dict(decoder="mwpf(cluster_node_limit=1000)"),
    dict(decoder="bposd"),
    dict(decoder="bposd(max_iter=5)"),
    dict(decoder="bposd(max_iter=5,osd_order=10,osd_method=osd_e)"),
    dict(decoder="bplsd"),
    dict(decoder="bplsd(max_iter=5)"),
    dict(decoder="bpuf"),
    dict(decoder="bpuf(max_iter=5)"),
    dict(decoder="pymatching", max_errors=1000000),
]

for custom in custom_kwargs:
    kwargs = { **default_kwargs, **custom }
    stats = logical_error_rate(**kwargs)
    print(f"{kwargs['decoder']}: {stats}, duration = {stats.duration // 60} min")

mwpf: Stats{ pL = 10415/141988249= 7.3(2)e-05, speed=9.46e-06s/S }, duration = 22.0 min
huf: Stats{ pL = 11731/158684569= 7.4(2)e-05, speed=8.51e-06s/S }, duration = 22.0 min
mwpf(cluster_node_limit=1000): Stats{ pL = 10868/148964761= 7.3(2)e-05, speed=9.13e-06s/S }, duration = 22.0 min
bposd: Stats{ pL = 10997/125032857= 8.8(2)e-05, speed=7.63e-06s/S }, duration = 15.0 min
bposd(max_iter=5): Stats{ pL = 10349/118183321= 8.8(2)e-05, speed=7.95e-06s/S }, duration = 15.0 min
bposd(max_iter=5,osd_order=10,osd_method=osd_e): Stats{ pL = 10047/136994201= 7.3(2)e-05, speed=9.78e-06s/S }, duration = 22.0 min
bplsd: Stats{ pL = 10805/125996235= 8.6(2)e-05, speed=8.65e-06s/S }, duration = 18.0 min
bplsd(max_iter=5): Stats{ pL = 13632/154425753= 8.8(2)e-05, speed=8.71e-06s/S }, duration = 22.0 min
bpuf: Stats{ pL = 10618/111846809= 9.5(2)e-05, speed=8.65e-06s/S }, duration = 16.0 min
bpuf(max_iter=5): Stats{ pL = 10726/114816409= 9.3(2)e-05, speed=8.49e-06s/S }, duration = 16.0 min
pymatching: S

## Circuit-level Color Code, $d=3, r=3, p=0.001$

In [7]:
# default_kwargs = {
    **global_kwargs,
    **dict(
        code = CodeCli("color(d=3,p=0.001)"),
        max_errors = 10000,
        save_resume_filepath = resume_filepath("color"),
    )
}
custom_kwargs = [
    dict(decoder="mwpf"),
    dict(decoder="huf"),
    dict(decoder="mwpf(cluster_node_limit=1000)"),
    dict(decoder="bposd"),
    dict(decoder="bposd(max_iter=5)"),
    dict(decoder="bposd(max_iter=5,osd_order=10,osd_method=osd_e)"),
    dict(decoder="bplsd"),
    dict(decoder="bplsd(max_iter=5)"),
    dict(decoder="bpuf"),
    dict(decoder="bpuf(max_iter=5)"),
    dict(decoder="bposd(max_iter=5,osd_order=15,osd_method=osd_e)"),
]

for custom in custom_kwargs:
    kwargs = { **default_kwargs, **custom }
    stats = logical_error_rate(**kwargs)
    print(f"{kwargs['decoder']}: {stats}, duration = {stats.duration // 60} min")

mwpf: Stats{ pL = 12378/4480409= 0.00276(6), speed=2.14e-05s/S }, duration = 1.0 min
huf: Stats{ pL = 11606/3926425= 0.00296(7), speed=1.25e-05s/S }, duration = 0.0 min
mwpf(cluster_node_limit=1000): Stats{ pL = 12399/4476313= 0.00277(6), speed=2.16e-05s/S }, duration = 1.0 min
bposd: Stats{ pL = 12702/785817= 0.0162(4), speed=1.15e-05s/S }, duration = 0.0 min
bposd(max_iter=5): Stats{ pL = 11095/710041= 0.0156(4), speed=9.11e-06s/S }, duration = 0.0 min
bposd(max_iter=5,osd_order=10,osd_method=osd_e): Stats{ pL = 11249/717209= 0.0157(4), speed=1.89e-05s/S }, duration = 0.0 min
bplsd: Stats{ pL = 10507/658841= 0.0159(4), speed=1.44e-05s/S }, duration = 0.0 min
bplsd(max_iter=5): Stats{ pL = 11477/732569= 0.0157(4), speed=1.27e-05s/S }, duration = 0.0 min
bpuf: Stats{ pL = 10843/676249= 0.0160(4), speed=1.43e-05s/S }, duration = 0.0 min
bpuf(max_iter=5): Stats{ pL = 11300/705945= 0.0160(4), speed=1.35e-05s/S }, duration = 0.0 min
bposd(max_iter=5,osd_order=15,osd_method=osd_e): Stats{ p